# CAFQA

In [1]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz
import numpy as np

d:\research-impl\clapton\clapton\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
np.random.seed(42)  # You can replace 42 with any integer seed
coeffs = np.random.random(len(paulis))
# coeffs = np.ones((len(paulis)))
print(coeffs)

In [3]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)

In [4]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False


In [5]:
# non-fixed gates will be optimized over
# RY and RZ gates can assume 4 values k = 0,1,2,3 which descripe multiples of pi/2

In [6]:
# the initial parameters are all 0
vqe_pcirc.read()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [7]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@-I-I---
         | |
q1: -I-I-|-X-@-I-I-
         |   |
q2: -I-I-@---X-I-I-

In [8]:
# we can assign a different set of parameters
vqe_pcirc.assign([0,1,2,3,0,1,2,3,0,1,2,3])

In [9]:
vqe_pcirc.stim_circuit().diagram()

q0: -I------S_DAG-X-@-Y-S----------------
                  | |
q1: -SQRT_Y-I-----|-X-@-SQRT_Y_DAG-Z-----
                  |   |
q2: -Y------S-----@---X-I----------S_DAG-

In [10]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id None with 1 procs



d:\research-impl\clapton\clapton\venv\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


[0, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([0, 3, 2, 2, 2, 0, 3, 0, 0, 1, 0, 2], dtype=object)]
[1, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([1, 3, 2, 2, 2, 0, 3, 0, 0, 1, 0, 2], dtype=object)]
[2, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([1, 3, 2, 2, 2, 0, 3, 0, 0, 1, 0, 2], dtype=object)]
[3, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([2, 3, 2, 2, 2, 0, 3, 0, 0, 1, 0, 2], dtype=object)]
[4, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([2, 3, 2, 2, 2, 0, 3, 0, 0, 1, 0, 2], dtype=object)]
[5, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([2, 3, 2, 2, 2, 0, 3, 0, 0, 1, 0, 2], dtype=object)]
[6, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([2, 3, 0, 2, 2, 2, 3, 1, 3, 1, 3, 3], dtype=object)]
[7, array([-3.41073462, -1.70536731, -1.70536731,  0.        ]), array([2, 3, 0, 2, 2, 2, 3, 1, 3, 1, 3, 3], dtype=object)]
[8, arra

In [11]:
# the best parameters are
ks_best

[1, 3, 2, 2, 2, 0, 3, 0, 0, 2, 0, 2]

In [12]:
# with energy
energy_best

np.float64(-1.7053673109431555)

In [13]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.stim_circuit().diagram()

q0: -SQRT_Y-----Z-X-@-SQRT_Y_DAG-Z---
                  | |
q1: -SQRT_Y_DAG-Z-|-X-@----------I-I-
                  |   |
q2: -Y----------I-@---X----------I-Z-

# noisy CAFQA (nCAFQA)

In [14]:
from clapton.depolarization import GateGeneralDepolarizationModel

In [15]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.02)

In [16]:
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)
vqe_pcirc.add_depolarization_model(nm)

In [17]:
# after every gate a depol channel is added
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)---I-----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                 | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|-----------------X-DEPOLARIZE2(0.02)-@-DEPOLARIZE2(0.02)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------
                                                                                                                         | |                                     | |
q2: ----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-@-DEPOLARIZE2(0.02)---------------------X-DEPOLARIZE2(0.02)--------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-

In [18]:
# we can perform nCAFQA by using the main optimization function "claptonize"
# now with the noisy circuit
# this is slower, as the noisy circuit needs to be sampled
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id None with 1 procs

[0, array([-5.6994, -2.6994, -3.    ,  0.    ]), array([0, 3, 0, 0, 0, 0, 2, 0, 2, 2, 1, 3], dtype=object)]
[1, array([-3.7418, -1.7418, -2.    ,  0.    ]), array([3, 1, 0, 1, 3, 0, 2, 3, 1, 2, 2, 2], dtype=object)]
[2, array([-5.675, -2.675, -3.   ,  0.   ]), array([0, 1, 0, 2, 2, 1, 1, 0, 2, 0, 1, 3], dtype=object)]
[3, array([-5.7066, -2.7066, -3.    ,  0.    ]), array([0, 3, 0, 0, 0, 0, 2, 0, 2, 2, 1, 3], dtype=object)]
[4, array([-5.6468, -2.6468, -3.    ,  0.    ]), array([1, 3, 0, 0, 2, 1, 1, 1, 3, 1, 2, 2], dtype=object)]
[5, array([-5.7456, -2.7456, -3.    ,  0.    ]), array([0, 1, 0, 2, 2, 1, 1, 0, 2, 0, 1, 3], dtype=object)]
[6, array([-5.7086, -2.7086, -3.    ,  0.    ]), array([0, 1, 0, 2, 2, 1, 1, 0, 2, 2, 1, 3], dtype=object)]
[7, array([-5.7412, -2.7412, -3.    ,  0.    ]), array([3, 3, 2, 2, 0, 0, 3, 1, 1, 2, 0, 0], dtype=object)]
[8, array([-5.7002, -2.7002, -3.    ,  0.    ]), array([1, 3, 0, 0, 2, 1, 1, 0, 2, 0,

In [19]:
# the best parameters are
ks_best

[1, 1, 0, 0, 0, 2, 2, 1, 3, 3, 2, 2]

In [20]:
# with noisy/noiseless energy
energy_noisy, energy_noiseless

(np.float64(-2.698), np.float64(-3.0))

In [21]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -SQRT_Y-I-X-@-Y-S_DAG--------
              | |
q1: -SQRT_Y-I-|-X-@-SQRT_Y-----Z-
              |   |
q2: -I------Z-@---X-SQRT_Y_DAG-Z-